In [ ]:

import gdown

url = 'https://app.roboflow.com/ds/Nu4o8sn8Ds?key=v4EID8IxBn'
output='smart.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://app.roboflow.com/ds/Nu4o8sn8Ds?key=v4EID8IxBn
To: /content/smart.zip
100%|██████████| 1.87G/1.87G [01:24<00:00, 22.2MB/s]


'smart.zip'

In [ ]:
!mkdir /content/datasets

In [ ]:
import os
os.chdir('/content/')  #change dir
!unzip -u smart.zip -d datasets/Visual-Pollution-1

In [ ]:
!nvidia-smi

Sat Jan 21 02:44:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    50W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# Pip install (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
%cd {HOME}
#you need to open the data.yaml file inorder to fix train and valid paths before you run training
#!yolo task=detect mode=train model=yolov8x.pt data=/content/datasets/Visual-Pollution-1/data.yaml epochs=50 imgsz=960 batch=24

/content
Ultralytics YOLOv8.0.13 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (A100-SXM4-40GB, 40536MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/datasets/Visual-Pollution-1/data.yaml, epochs=50, patience=50, batch=24, imgsz=960, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, wo

In [ ]:
import gdown

url='https://docs.google.com/uc?export=download&id=1_lvjNOUjM_rQYHNpzcuO_QsbTqELWbMs&confirm=t'
output='dataset.zip'
gdown.download(url, output, quiet=False) #, use_cookies=True)


Downloading...
From: https://docs.google.com/uc?export=download&id=1_lvjNOUjM_rQYHNpzcuO_QsbTqELWbMs&confirm=t
To: /content/dataset.zip
100%|██████████| 2.34G/2.34G [00:10<00:00, 219MB/s]


'dataset.zip'

In [ ]:
import gdown
url='https://docs.google.com/uc?export=download&id=1Zof-imhUe7CYTmjMcJF4Rq6Hz-t8GL_7&confirm=t'
output='yolo8_trained_model.pt'
gdown.download(url, output, quiet=False) #, use_cookies=True)


In [ ]:
import os
os.chdir('/content/') 
!unzip -u dataset.zip -d dataset/

In [ ]:
import pandas as pd
test_df = pd.read_csv("/content/dataset/dataset/test.csv")
test_df

In [ ]:
from tqdm import tqdm

count_test = 0 

smart_dict={'GRAFFITI' : 0.0 , 'FADED_SIGNAGE': 1.0 , 'POTHOLES': 2.0,
            'GARBAGE' : 3.0 , 'CONSTRUCTION_ROAD': 4.0 , 'BROKEN_SIGNAGE': 5.0,
            'BAD_STREETLIGHT' : 6.0 , 'BAD_BILLBOARD': 7.0 , 'SAND_ON_ROAD':8.0,
            'CLUTTER_SIDEWALK' : 9.0 , 'UNKEPT_FACADE': 10.0}

model = YOLO('/content/yolo8_trained_model.pt')


sub_df = pd.DataFrame(columns=['class','image_path','name','xmax','xmin','ymax','ymin'])
for index, row in tqdm(test_df.iterrows()):  
  t_path = '/content/dataset/dataset/images/'+row["image_path"]
  outputs = model.predict(source=t_path, save=False, save_txt=False) 
  pred_boxes = outputs[0].boxes
  
  if len(pred_boxes)>0:
    one_pass = False

    for idx in range(len(pred_boxes)):
      xmin, ymin, xmax, ymax = pred_boxes[idx].xyxy[0].cpu().numpy()
      cls = pred_boxes[idx].cls[0].cpu().numpy()
      name = str(model.model.names[int(cls)])
      score = pred_boxes[idx].conf[0].cpu().numpy()
      if name == "None":
         continue
      pclass = smart_dict[name]
      image_path = row["image_path"]
      

      if score >= 0.0:
      
        xmin = xmin/2
        ymin = ymin/2
        xmax = xmax/2
        ymax = ymax/2
        


        sub_df = sub_df.append({'class':pclass, 'image_path':image_path, 'name':name, 'xmax':xmax, 'xmin':xmin, 'ymax':ymax, 'ymin':ymin}, ignore_index=True)
        one_pass = True

    if one_pass == False:
      sub_df = sub_df.append({'class':3.0, 'image_path':row["image_path"], 'name':"GARBAGE", 'xmax':1260, 'xmin':660, 'ymax':660, 'ymin':420}, ignore_index=True)
      count_test +=1
  else:
    sub_df = sub_df.append({'class':3.0, 'image_path':row["image_path"], 'name':"GARBAGE", 'xmax':1260, 'xmin':660, 'ymax':660, 'ymin':420}, ignore_index=True)
    count_test +=1

0it [00:00, ?it/s]Ultralytics YOLOv8.0.13 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (A100-SXM4-40GB, 40536MiB)
Fusing... 
Model summary: 268 layers, 68133198 parameters, 0 gradients, 257.4 GFLOPs
2092it [01:58, 17.59it/s]


In [ ]:
sub_df

,class,image_path,name,xmax,xmin,ymax,ymin
0,7.0,953ab1447c46ecfef67ab14629cd70c7.jpg,BAD_BILLBOARD,705.5,607.5,225.5,172.0
1,3.0,953ab1447c46ecfef67ab14629cd70c7.jpg,GARBAGE,443.5,394.5,301.0,262.0
2,3.0,e4ddbaa7970fca225a51288ce5f7d3f9.jpg,GARBAGE,641.0,498.5,412.5,333.0
3,2.0,5b8120d69607a077b7583334be3ba18b.jpg,POTHOLES,758.0,533.5,527.5,339.5
4,2.0,5b8120d69607a077b7583334be3ba18b.jpg,POTHOLES,647.5,490.0,396.5,281.0
...,...,...,...,...,...,...,...
6180,3.0,15afe8bce6c71deed2d53f56b78b90a8.jpg,GARBAGE,815.0,771.5,330.0,286.5
6181,9.0,984841724143c66506986b3faa50e4b9.jpg,CLUTTER_SIDEWALK,668.5,17.0,338.0,196.5
6182,9.0,984841724143c66506986b3faa50e4b9.jpg,CLUTTER_SIDEWALK,864.5,587.0,318.5,199.0
6183,9.0,984841724143c66506986b3faa50e4b9.jpg,CLUTTER_SIDEWALK,960.0,729.5,326.0,184.5


In [ ]:
sub_df.to_csv("/content/sub_y8v1.csv",index=False)